In [8]:
import pandas as pd
import random
import os
import numpy as np

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

from tensorflow import keras
from sklearn.model_selection import train_test_split

In [133]:
class CFG:
    SEED = 42

In [134]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

In [135]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class','father','mother','gender'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id','father','mother','gender'])
        return df_x

train_x, train_y = get_x_y(train)
test_x = get_x_y(test)
train_x

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A
1,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A
2,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A
3,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G
4,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2,A G,A G,A A,G A,C C,A G,A A,G A,A A,G G,A G,G A,A A,A A,A A
258,2,G G,A A,C A,A A,A A,A G,G A,G A,A A,A G,A G,A A,A G,A A,G A
259,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A A,G G,G G,G G,C A,G G
260,1,A A,G G,A A,G A,A A,G G,G G,A A,G A,A G,A G,G A,G G,C A,G G


In [136]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)

LabelEncoder()

In [137]:
for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

In [138]:
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2)

In [139]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(209, 16) (53, 16) (209,) (53,)


In [143]:
model = keras.Sequential()
model.add(keras.layers.Dense(units=100, activation='sigmoid', input_shape=(16,)))
model.add(keras.layers.Dense(units=100, activation='softmax'))
model.add(keras.layers.Dense(units=3, activation='softmax'))


In [144]:
model.summary()


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_82 (Dense)            (None, 100)               1700      
                                                                 
 dense_83 (Dense)            (None, 100)               10100     
                                                                 
 dense_84 (Dense)            (None, 3)                 303       
                                                                 
Total params: 12,103
Trainable params: 12,103
Non-trainable params: 0
_________________________________________________________________


In [145]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [146]:
model.fit(X_train, y_train, epochs=300)

Epoch 1/300
7/7 [==============================] - 1s 5ms/step - loss: 1.0921 - accuracy: 0.4354
Epoch 2/300
7/7 [==============================] - 0s 6ms/step - loss: 1.0829 - accuracy: 0.4354
Epoch 3/300
7/7 [==============================] - 0s 5ms/step - loss: 1.0739 - accuracy: 0.4354
Epoch 4/300
7/7 [==============================] - 0s 5ms/step - loss: 1.0643 - accuracy: 0.4354
Epoch 5/300
7/7 [==============================] - 0s 4ms/step - loss: 1.0523 - accuracy: 0.4354
Epoch 6/300
7/7 [==============================] - 0s 5ms/step - loss: 1.0362 - accuracy: 0.4498
Epoch 7/300
7/7 [==============================] - 0s 5ms/step - loss: 1.0165 - accuracy: 0.6172
Epoch 8/300
7/7 [==============================] - 0s 5ms/step - loss: 0.9945 - accuracy: 0.6938
Epoch 9/300
7/7 [==============================] - 0s 6ms/step - loss: 0.9705 - accuracy: 0.6986
Epoch 10/300
7/7 [==============================] - 0s 7ms/step - loss: 0.9468 - accuracy: 0.6986
Epoch 11/300
7/7 [===========

7/7 [==============================] - 0s 5ms/step - loss: 0.4249 - accuracy: 0.9809
Epoch 85/300
7/7 [==============================] - 0s 6ms/step - loss: 0.4210 - accuracy: 0.9809
Epoch 86/300
7/7 [==============================] - 0s 4ms/step - loss: 0.4171 - accuracy: 0.9809
Epoch 87/300
7/7 [==============================] - 0s 4ms/step - loss: 0.4138 - accuracy: 0.9809
Epoch 88/300
7/7 [==============================] - 0s 4ms/step - loss: 0.4099 - accuracy: 0.9809
Epoch 89/300
7/7 [==============================] - 0s 4ms/step - loss: 0.4065 - accuracy: 0.9809
Epoch 90/300
7/7 [==============================] - 0s 4ms/step - loss: 0.4025 - accuracy: 0.9809
Epoch 91/300
7/7 [==============================] - 0s 4ms/step - loss: 0.3991 - accuracy: 0.9809
Epoch 92/300
7/7 [==============================] - 0s 4ms/step - loss: 0.3958 - accuracy: 0.9809
Epoch 93/300
7/7 [==============================] - 0s 4ms/step - loss: 0.3922 - accuracy: 0.9809
Epoch 94/300
7/7 [===============

7/7 [==============================] - 0s 5ms/step - loss: 0.2175 - accuracy: 0.9904
Epoch 168/300
7/7 [==============================] - 0s 5ms/step - loss: 0.2161 - accuracy: 0.9904
Epoch 169/300
7/7 [==============================] - 0s 3ms/step - loss: 0.2147 - accuracy: 0.9904
Epoch 170/300
7/7 [==============================] - 0s 5ms/step - loss: 0.2133 - accuracy: 0.9904
Epoch 171/300
7/7 [==============================] - 0s 4ms/step - loss: 0.2118 - accuracy: 0.9904
Epoch 172/300
7/7 [==============================] - 0s 4ms/step - loss: 0.2105 - accuracy: 0.9904
Epoch 173/300
7/7 [==============================] - 0s 4ms/step - loss: 0.2091 - accuracy: 0.9904
Epoch 174/300
7/7 [==============================] - 0s 4ms/step - loss: 0.2076 - accuracy: 0.9904
Epoch 175/300
7/7 [==============================] - 0s 8ms/step - loss: 0.2064 - accuracy: 0.9904
Epoch 176/300
7/7 [==============================] - 0s 8ms/step - loss: 0.2054 - accuracy: 0.9904
Epoch 177/300
7/7 [=====

7/7 [==============================] - 0s 4ms/step - loss: 0.1216 - accuracy: 0.9952
Epoch 250/300
7/7 [==============================] - 0s 5ms/step - loss: 0.1207 - accuracy: 0.9952
Epoch 251/300
7/7 [==============================] - 0s 5ms/step - loss: 0.1203 - accuracy: 0.9952
Epoch 252/300
7/7 [==============================] - 0s 8ms/step - loss: 0.1189 - accuracy: 0.9952
Epoch 253/300
7/7 [==============================] - 0s 8ms/step - loss: 0.1187 - accuracy: 0.9952
Epoch 254/300
7/7 [==============================] - 0s 3ms/step - loss: 0.1190 - accuracy: 0.9952
Epoch 255/300
7/7 [==============================] - 0s 3ms/step - loss: 0.1159 - accuracy: 0.9952
Epoch 256/300
7/7 [==============================] - 0s 3ms/step - loss: 0.1159 - accuracy: 0.9952
Epoch 257/300
7/7 [==============================] - 0s 4ms/step - loss: 0.1141 - accuracy: 1.0000
Epoch 258/300
7/7 [==============================] - 0s 3ms/step - loss: 0.1137 - accuracy: 0.9952
Epoch 259/300
7/7 [=====

In [147]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 28s 4ms/step - loss: 0.1752 - accuracy: 0.9434


[0.17515826225280762, 0.9433962106704712]

In [149]:
preds = model.predict(test_x)

6/6 [==============================] - 0s 3ms/step


In [150]:
print(preds)

[[0.9170731  0.04871092 0.03421603]
 [0.0295132  0.9321395  0.03834729]
 [0.02541832 0.0550203  0.91956127]
 [0.02598817 0.05742934 0.9165826 ]
 [0.91710204 0.04869876 0.03419913]
 [0.02956234 0.93193793 0.03849973]
 [0.02545301 0.05505294 0.9194941 ]
 [0.02967872 0.9314595  0.0388617 ]
 [0.9170732  0.04871061 0.03421622]
 [0.9171002  0.04869927 0.03420054]
 [0.02541058 0.05501302 0.9195764 ]
 [0.02950094 0.9321981  0.03830096]
 [0.05995657 0.6481027  0.29194057]
 [0.9170921  0.04870269 0.03420527]
 [0.03284989 0.91744196 0.04970829]
 [0.02964346 0.93160295 0.03875367]
 [0.9170827  0.04870665 0.03421075]
 [0.02955284 0.93197674 0.03847045]
 [0.02950599 0.932169   0.03832492]
 [0.03509071 0.103836   0.8610733 ]
 [0.02948984 0.9322341  0.03827611]
 [0.02949296 0.9322214  0.03828553]
 [0.0296563  0.9315565  0.03878721]
 [0.9170628  0.04871493 0.03422217]
 [0.02957282 0.9318975  0.03852969]
 [0.05356912 0.24865614 0.6977747 ]
 [0.02959417 0.93180597 0.03859986]
 [0.9170675  0.04871303 0.03

In [8]:
clf = RandomForestClassifier(random_state=CFG.SEED)
clf.fit(train_x, train_y)

RandomForestClassifier(random_state=42)

In [9]:
preds = clf.predict(test_x)
print('Done.')
print(preds)

Done.
[0 1 2 1 0 1 2 1 0 0 2 1 2 0 1 1 0 1 1 2 1 1 1 0 1 1 1 0 0 1 0 0 1 2 0 1 2
 1 1 2 0 1 2 1 1 1 1 2 1 2 0 1 0 1 1 1 2 0 1 2 0 1 2 2 2 0 1 0 0 1 1 1 0 0
 2 1 2 1 1 1 2 1 0 1 1 1 1 1 2 0 1 1 2 1 1 2 0 1 0 2 0 1 1 2 0 0 2 1 0 1 2
 1 1 1 1 0 0 2 1 1 0 1 1 2 2 1 2 1 0 1 0 0 1 1 1 2 0 0 1 0 0 0 2 1 1 1 0 1
 2 0 0 1 0 1 1 0 0 1 2 0 1 2 2 1 0 0 2 1 1 0 1 2 2 1 1]


In [112]:
submit = pd.read_csv('./sample_submission.csv')

submit['class'] = class_le.inverse_transform(preds)

submit.to_csv('./answer/submitdr.csv', index=False)

ValueError: y should be a 1d array, got an array of shape (175, 3) instead.